In [53]:
import click
from io import BytesIO
import pandas as pd
import pyarrow.parquet as pq
import requests
from tqdm.auto import tqdm # tracks ingestion progress
from sqlalchemy import create_engine

In [4]:
# Getting data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet

--2026-01-17 16:40:00--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.171.57.179, 3.171.57.103, 3.171.57.69, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.171.57.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1164775 (1.1M) [binary/octet-stream]
Saving to: ‘green_tripdata_2025-11.parquet’

green_tripdata_2025 100%[===================>]   1.11M  7.37MB/s    in 0.2s    

2026-01-17 16:40:01 (7.37 MB/s) - ‘green_tripdata_2025-11.parquet’ saved [1164775/1164775]

--2026-01-17 16:40:01--  http://./resources/
Resolving . (.)... failed: Name or service not known.
wget: unable to resolve host address ‘.’
FINISHED --2026-01-17 16:40:01--
Total wall clock time: 0.6s
Downloaded: 1 files, 1.1M in 0.2s (7.37 MB/s)


In [5]:
!wget -P ./resources/ https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

--2026-01-17 16:41:09--  https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/513814948/5a2cc2f5-b4cd-4584-9c62-a6ea97ed0e6a?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-17T22%3A31%3A10Z&rscd=attachment%3B+filename%3Dtaxi_zone_lookup.csv&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-17T21%3A30%3A30Z&ske=2026-01-17T22%3A31%3A10Z&sks=b&skv=2018-11-09&sig=1Xr8oYc07POmvxf6qJdRbunaPc6FI4%2FTiifKGZ%2B78Ls%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2ODY4NjM3MCwibmJmIjoxNzY4Njg2MDcwLCJwYXRoIjoicmVsZWFzZWFzc2V

In [2]:
# Parameters for sql engine
pg_user = 'postgres'
pg_pass = 'postgres'
pg_host = 'localhost'
pg_port = 5432
pg_db = 'ny_taxi'
year = 2025
month = 11
target_table = 'green_taxi_data'
chunksize = 100000

In [3]:
df = pd.read_parquet('./resources/green_tripdata_2025-11.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [4]:
df.shape

(46912, 21)

In [5]:
df.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [57]:
dtype = {
    "VendorID": "int32",
    "store_and_fwd_flag": "object",
    "RatecodeID": "float64",
    "PULocationID": "int32",
    "DOLocationID": "int32",
    "passenger_count": "float64",
    "trip_distance": "float64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "payment_type": "float64",
    "trip_type": "float64",
    "congestion_surcharge": "float64",
    "cbd_congestion_fee": "float64"
}

parse_dates = [
    "lpep_pickup_datetime",
    "lpep_dropoff_datetime"
]

In [13]:
for column in df.columns:
    print(f'Column: {column} /n result: {df[column].unique()}')

Column: VendorID /n result: [2 1 6]
Column: lpep_pickup_datetime /n result: <DatetimeArray>
['2025-11-01 00:34:48', '2025-11-01 00:18:52', '2025-11-01 01:03:14',
 '2025-11-01 00:10:57', '2025-11-01 00:03:48', '2025-11-01 00:42:13',
 '2025-11-01 00:05:41', '2025-11-01 00:42:14', '2025-11-01 00:03:08',
 '2025-11-01 00:56:33',
 ...
 '2025-11-30 17:45:00', '2025-11-30 17:22:37', '2025-11-30 18:01:00',
 '2025-11-30 18:01:25', '2025-11-30 18:31:00', '2025-11-30 19:58:34',
 '2025-11-30 19:34:00', '2025-11-30 21:46:46', '2025-11-30 21:00:00',
 '2025-11-30 23:26:00']
Length: 46123, dtype: datetime64[us]
Column: lpep_dropoff_datetime /n result: <DatetimeArray>
['2025-11-01 00:41:39', '2025-11-01 00:24:27', '2025-11-01 01:15:24',
 '2025-11-01 00:24:53', '2025-11-01 00:19:38', '2025-11-01 01:04:50',
 '2025-11-01 00:39:20', '2025-11-01 01:13:20', '2025-11-01 00:06:27',
 '2025-11-01 01:01:34',
 ...
 '2025-11-30 18:59:00', '2025-11-30 18:09:41', '2025-11-30 18:16:00',
 '2025-11-30 18:37:16', '2025-11

In [18]:
prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
url = f'{prefix}/green_tripdata_{year}-{month:02d}.parquet'
print(url)

https://d37ci6vzurychx.cloudfront.net/trip-data//green_tripdata_2025-11.parquet


In [64]:
def ingest_data(pg_user, pg_pass, pg_host, pg_port, pg_db, year, month, target_table, chunksize):
    # Ingestion logic here
    # Data source
    prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data'
    url = f'{prefix}/green_tripdata_{year}-{month:02d}.parquet'

    # Download parquet file
    response = requests.get(url, stream=True)
    response.raise_for_status()

    # Read the Parquet file from bytes
    table = pq.read_table(BytesIO(response.content))
        
    # Define sql engine and database
    engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')
    

    first = True
    for batch in tqdm(table):
        df_chunk= batch.to_pandas()
        if first:
            # Create table schema (no data)
            df_chunk.head(0).to_sql(
                name=target_table,
                con=engine,
                if_exists="replace"
            )
            first = False
            print("Table created")
        
        # Insert chunk
        df_chunk.to_sql(
            name=target_table,
            con=engine,
            if_exists="append",
            chunksize=chunksize
        )

        print(f"Inserted:, {len(df_chunk)} rows")    

In [65]:
ingest_data(pg_user, pg_pass, pg_host, pg_port, pg_db, year, month, target_table, chunksize)

  0%|          | 0/46912 [00:00<?, ?it/s]

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [67]:
def ingest_data(pg_user, pg_pass, pg_host, pg_port, pg_db, year, month, target_table, chunksize):
    # Data source
    prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data'
    url = f'{prefix}/green_tripdata_{year}-{month:02d}.parquet'

    # Download parquet file in memory
    response = requests.get(url, stream=True)
    response.raise_for_status()

    # Read the Parquet file from bytes
    table = pq.read_table(BytesIO(response.content))

    # Define SQL engine
    engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')

    # Convert table to batches and iterate
    first = True
    for batch in tqdm(table.to_batches(), desc="Processing batches"):
        df_chunk = batch.to_pandas()

        if first:
            # Create table schema (no data)
            df_chunk.head(0).to_sql(
                name=target_table,
                con=engine,
                if_exists="replace",
                index=False
            )
            first = False
            print("Table created")

        # Insert chunk
        df_chunk.to_sql(
            name=target_table,
            con=engine,
            if_exists="append",
            chunksize=chunksize,
            index=False
        )

        print(f"Inserted {len(df_chunk)} rows")

In [68]:
ingest_data(pg_user, pg_pass, pg_host, pg_port, pg_db, year, month, target_table, chunksize)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)